# Imports

In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install -q accelerate -U
!pip install -q simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 70.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.5/258.5 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 64

In [4]:
cd '/content/drive/My Drive/Colab Notebooks/NLP'

/content/drive/My Drive/Colab Notebooks/NLP


In [5]:
import os
import wandb

# Disable wandb authorization request
os.environ["WANDB_START_METHOD"]="thread"
wandb.init(mode="disabled")

In [6]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs, MultiLabelClassificationModel, MultiLabelClassificationArgs
from urllib import request
import pandas as pd
import logging
import torch
from collections import Counter
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from ast import literal_eval
import numpy as np
from dont_patronize_me import DontPatronizeMe
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from transformers import DebertaTokenizer, DebertaForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch
import torch.functional as F
import tqdm

In [7]:
random_seed = 42

In [8]:
# prepare logger
logging.basicConfig(level=logging.INFO)

transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# check gpu
cuda_available = torch.cuda.is_available()

device = 'cuda' if cuda_available else 'cpu'

print('Cuda available?', cuda_available)

Cuda available? True


# Data loading

In [9]:
def load_data(train_size=0.8, random_state=random_seed):
    dpm = DontPatronizeMe('./data', './data')
    dpm.load_task1()
    trids = pd.read_csv('data/train_semeval_parids-labels.csv')
    teids = pd.read_csv('data/dev_semeval_parids-labels.csv')
    trids.par_id = trids.par_id.astype(str)
    teids.par_id = teids.par_id.astype(str)
    data = dpm.train_task1_df

    rows_train_val = [] # will contain par_id, label and text
    for idx in range(len(trids)):
        parid = trids.par_id[idx]
        # select row from original dataset to retrieve `text` and binary label
        instance = data.loc[data.par_id == parid]
        keyword = instance.keyword.values[0]
        text = instance.text.values[0]
        label = instance.label.values[0]
        rows_train_val.append({
            'par_id':parid,
            'community':keyword,
            'text':text,
            'label':label
        })

    rows_train, rows_val = train_test_split(rows_train_val, train_size=train_size, random_state=random_state)

    rows_test = [] # will contain par_id, label and text
    for idx in range(len(teids)):
        parid = teids.par_id[idx]
        #print(parid)
        # select row from original dataset
        instance = data.loc[data.par_id == parid]
        keyword = instance.keyword.values[0]
        text = instance.text.values[0]
        label = instance.label.values[0]
        rows_test.append({
            'par_id':parid,
            'community':keyword,
            'text':text,
            'label':label
        })
    print(len(rows_train), len(rows_val), len(rows_test))

    return pd.DataFrame(rows_train), pd.DataFrame(rows_val), pd.DataFrame(rows_test)

train_dataset_raw, eval_dataset_raw, test_dataset_raw = load_data(train_size=0.8)

6700 1675 2094


# Encode data into a Dataset

In [10]:
# Load the DeBERTa tokenizer
tokenizer = DebertaTokenizer.from_pretrained("microsoft/deberta-base")

max_length = 192
batch_size = 32

train_text = train_dataset_raw.text.values
eval_text = eval_dataset_raw.text.values
test_text = test_dataset_raw.text.values

encoding_train = tokenizer(train_text.tolist(), return_tensors='pt', padding="max_length", truncation=True, max_length=max_length)
encoding_eval = tokenizer(eval_text.tolist(), return_tensors='pt', padding="max_length", truncation=True, max_length=max_length)
encoding_test = tokenizer(test_text.tolist(), return_tensors='pt', padding="max_length", truncation=True, max_length=max_length)

encoding_train['label'] = torch.tensor([[0,1] if x == 1 else [1,0] for x in train_dataset_raw['label'].tolist()], dtype=torch.float32)
encoding_eval['label'] = torch.tensor([[0,1] if x == 1 else [1,0] for x in eval_dataset_raw['label'].tolist()], dtype=torch.float32)
encoding_test['label'] = torch.tensor([[0,1] if x == 1 else [1,0] for x in test_dataset_raw['label'].tolist()], dtype=torch.float32)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/474 [00:00<?, ?B/s]

In [11]:
class CustomDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        train_input_ids = self.encodings.input_ids[idx]
        train_token_type_ids = self.encodings.token_type_ids[idx]
        train_attention_mask = self.encodings.attention_mask[idx]
        train_labels = self.encodings.label[idx]
        return {
            'input_ids': train_input_ids,
            'token_type_ids': train_token_type_ids,
            'attention_mask': train_attention_mask,
            'labels': train_labels
        }

    def __len__(self):
        return len(self.encodings.input_ids)

In [12]:
# Create an instance of the CustomDataset class
train_dataset = CustomDataset(encoding_train)
eval_dataset = CustomDataset(encoding_eval)
test_dataset = CustomDataset(encoding_test)

# Model loading and training with custom trainer loss



In [13]:
model = DebertaForSequenceClassification.from_pretrained("microsoft/deberta-base", num_labels=2)
model = model.to(device)

pytorch_model.bin:   0%|          | 0.00/559M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
def f1_loss(_y_pred, _y_true):
    y_pred = _y_pred[:,1]
    y_true = _y_true[:,1]
    tp = torch.sum((y_true * y_pred).float(), dim=0)
    #tn = torch.sum(((1 - y_true) * (1 - y_pred)).float(), dim=0)
    fp = torch.sum(((1 - y_true) * y_pred).float(), dim=0)
    fn = torch.sum((y_true * (1 - y_pred)).float(), dim=0)

    p = tp / (tp + fp + torch.tensor(1e-7))  # Add epsilon
    r = tp / (tp + fn + torch.tensor(1e-7))  # Add epsilon

    f1 = 2 * p * r / (p + r + torch.tensor(1e-7))  # Add epsilon
    #f1 = torch.where(torch.isnan(f1), torch.zeros_like(f1), f1)
    return 1 - f1

In [15]:
class PatronizeTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs = False):
        """
        How the loss is computed by Trainer. By default, all models return the loss in the first element.

        Subclass and override for custom behavior.
        """
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        predictions = torch.nn.functional.softmax(logits, dim=1)
        loss = f1_loss(predictions, labels)

        return (loss, outputs) if return_outputs else loss

In [16]:
epochs = 30
lr = 1e-5

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    labels = np.argmax(labels, axis=-1)
    f1 = f1_score(labels, predictions, average='binary')
    return {'f1': f1}

trainingargs = TrainingArguments(
    learning_rate=lr,
    weight_decay=1e-2,
    output_dir='/content/training_results',
    overwrite_output_dir=True,
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    evaluation_strategy="epoch",
    report_to=None,
    metric_for_best_model="f1",
    save_strategy='epoch',
    load_best_model_at_end=True,
    seed=random_seed,
    optim='adamw_torch',
)

trainer = PatronizeTrainer(
    model=model,
    args=trainingargs,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()
# Here the best model has been already loaded
trainer.predict(test_dataset)

Epoch,Training Loss,Validation Loss,F1
1,No log,0.817314,0.442857
2,No log,0.523218,0.487342
3,0.712400,0.524576,0.515050
4,0.712400,0.540003,0.485915
5,0.548400,0.514328,0.531915
6,0.548400,0.554163,0.493056
7,0.548400,0.515205,0.505119
8,0.461200,0.579274,0.480836
9,0.461200,0.667429,0.410256
10,0.446800,0.560175,0.493333


PredictionOutput(predictions=array([[ 6.406392 , -7.0404124],
       [-5.8488603,  5.987648 ],
       [ 6.544084 , -7.1145153],
       ...,
       [ 6.4820485, -7.1485286],
       [ 4.796703 , -5.1913185],
       [ 6.5260696, -7.1678967]], dtype=float32), label_ids=array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32), metrics={'test_loss': 0.9122962951660156, 'test_f1': 0.5515873015873015, 'test_runtime': 26.9954, 'test_samples_per_second': 77.569, 'test_steps_per_second': 2.445})

## Custom trainer

In [18]:
trainer.save_model('./results/custom_loss_model')